<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_CNN_SHAP_whitelist_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
!pip install shap

In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load dataset & model

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [4]:
model_cnn = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_cnn_bin.h5', compile=True)

In [5]:
x_normal = df[df['Label'] == 0].reset_index(drop=True)

In [6]:
x_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54797 entries, 0 to 54796
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      54797 non-null  float64
 1   Tot Fwd Pkts       54797 non-null  float64
 2   Tot Bwd Pkts       54797 non-null  float64
 3   TotLen Fwd Pkts    54797 non-null  float64
 4   TotLen Bwd Pkts    54797 non-null  float64
 5   Fwd Pkt Len Max    54797 non-null  float64
 6   Fwd Pkt Len Min    54797 non-null  float64
 7   Fwd Pkt Len Mean   54797 non-null  float64
 8   Fwd Pkt Len Std    54797 non-null  float64
 9   Bwd Pkt Len Max    54797 non-null  float64
 10  Bwd Pkt Len Min    54797 non-null  float64
 11  Bwd Pkt Len Mean   54797 non-null  float64
 12  Bwd Pkt Len Std    54797 non-null  float64
 13  Flow Byts/s        54797 non-null  float64
 14  Flow Pkts/s        54797 non-null  float64
 15  Flow IAT Mean      54797 non-null  float64
 16  Flow IAT Std       547

In [7]:
x_normal['Label'].value_counts()

0    54797
Name: Label, dtype: int64

In [8]:
df = df.drop(columns=['Label'])

In [9]:
x_normal = x_normal.drop(columns=['Label'])

In [10]:
feature_list = x_normal.columns

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_cnn, shap.sample(df, 100))

In [12]:
normal_150 = x_normal.head(150)

shap_values takes 1g30' to calculate

In [13]:
shap_values = explainer.shap_values(normal_150)

  0%|          | 0/150 [00:00<?, ?it/s]

In [14]:
data_df = pd.DataFrame(shap_values[0])

In [20]:
len(shap_values)

1

In [21]:
len(shap_values[0])

150

In [22]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/CNN_HSJ/150_samples_[0].csv', index=False)

In [16]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,-0.000080,0.0,0.0,0.0,0.0,-0.000085,0.000000,-0.000091,0.000000,-0.000376,...,-0.000075,0.0,-0.000185,0.0,-0.000381,0.0,0.000000,-0.000923,0.000818,0.000000
1,0.000025,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000006,0.000003,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000017,0.000000,0.000000,0.000056
2,0.000034,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000016,0.000000,0.000000,0.000046
3,0.000028,0.0,0.0,0.0,0.0,0.000000,0.000004,0.000000,0.000005,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000010,0.000000,0.000000,0.000059
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [23]:
data_dict = {}
for idx in range(0, 150):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Normal sample ' + str(idx)] = row_values

In [24]:
print(data_dict)

{'Normal sample 0': ['FIN Flag Cnt', 'Idle Max', 'Bwd IAT Max', 'Pkt Len Max', 'Bwd Pkts/s', 'URG Flag Cnt', 'Fwd IAT Max', 'ACK Flag Cnt', 'SYN Flag Cnt', 'Bwd Seg Size Avg'], 'Normal sample 1': ['SYN Flag Cnt', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Bwd Pkts/s', 'Bwd PSH Flags', 'PSH Flag Cnt', 'Init Bwd Win Byts', 'Down/Up Ratio', 'Flow Pkts/s', 'URG Flag Cnt'], 'Normal sample 2': ['SYN Flag Cnt', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Bwd Pkts/s', 'Bwd PSH Flags', 'PSH Flag Cnt', 'Down/Up Ratio', 'Init Bwd Win Byts', 'URG Flag Cnt', 'Flow Pkts/s'], 'Normal sample 3': ['SYN Flag Cnt', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Bwd Pkts/s', 'Bwd PSH Flags', 'PSH Flag Cnt', 'Init Bwd Win Byts', 'Down/Up Ratio', 'URG Flag Cnt', 'Flow Pkts/s'], 'Normal sample 4': ['FIN Flag Cnt', 'Bwd Pkts/s', 'ACK Flag Cnt', 'Bwd PSH Flags', 'PSH Flag Cnt', 'URG Flag Cnt', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow IAT Max', 'Flow Duration'], 'Normal sample 5': ['SYN Flag Cnt', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Bwd Pkts/s', 'Bwd 

In [25]:
len(data_dict)

150

# New whitelist: 38 top features with over 10k freq each

In [26]:
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10'])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, Normal sample 0 to Normal sample 149
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Feature 1   150 non-null    object
 1   Feature 2   150 non-null    object
 2   Feature 3   150 non-null    object
 3   Feature 4   150 non-null    object
 4   Feature 5   150 non-null    object
 5   Feature 6   150 non-null    object
 6   Feature 7   150 non-null    object
 7   Feature 8   150 non-null    object
 8   Feature 9   150 non-null    object
 9   Feature 10  150 non-null    object
dtypes: object(10)
memory usage: 12.9+ KB


In [28]:
df.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10
Normal sample 0,FIN Flag Cnt,Idle Max,Bwd IAT Max,Pkt Len Max,Bwd Pkts/s,URG Flag Cnt,Fwd IAT Max,ACK Flag Cnt,SYN Flag Cnt,Bwd Seg Size Avg
Normal sample 1,SYN Flag Cnt,ACK Flag Cnt,FIN Flag Cnt,Bwd Pkts/s,Bwd PSH Flags,PSH Flag Cnt,Init Bwd Win Byts,Down/Up Ratio,Flow Pkts/s,URG Flag Cnt
Normal sample 2,SYN Flag Cnt,ACK Flag Cnt,FIN Flag Cnt,Bwd Pkts/s,Bwd PSH Flags,PSH Flag Cnt,Down/Up Ratio,Init Bwd Win Byts,URG Flag Cnt,Flow Pkts/s
Normal sample 3,SYN Flag Cnt,ACK Flag Cnt,FIN Flag Cnt,Bwd Pkts/s,Bwd PSH Flags,PSH Flag Cnt,Init Bwd Win Byts,Down/Up Ratio,URG Flag Cnt,Flow Pkts/s
Normal sample 4,FIN Flag Cnt,Bwd Pkts/s,ACK Flag Cnt,Bwd PSH Flags,PSH Flag Cnt,URG Flag Cnt,Bwd IAT Tot,Bwd URG Flags,Flow IAT Max,Flow Duration


In [29]:
df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/cnn_top10_of_each_normalfile.csv')

In [30]:
freq = {}
for column in df.columns:
  for feature in df[column]:
    if feature in freq:
      continue
    else:
      count = sum(df[column_2] == feature for column_2 in df.columns)
      freq[feature] = count.sum()
      print(freq[feature])

141
123
148
20
98
6
47
36
97
86
17
149
131
6
9
11
1
116
46
2
11
82
3
26
39
3
9
1
5
12
1
2
2
1
1
1
1
2
1
2
1
1
1
1
1


In [31]:
len(freq)

45

In [32]:
freq_df = pd.DataFrame(list(freq.items()), columns=['Feature', 'Frequency'])
freq_df.head()

,Feature,Frequency
0,FIN Flag Cnt,141
1,SYN Flag Cnt,123
2,Bwd Pkts/s,148
3,Bwd Pkt Len Max,20
4,ACK Flag Cnt,98


In [33]:
freq_df.sort_values(by=['Frequency'], ascending=False, inplace=True)
freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 11 to 44
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Feature    45 non-null     object
 1   Frequency  45 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [34]:
freq_df.head(24)

,Feature,Frequency
11,URG Flag Cnt,149
2,Bwd Pkts/s,148
0,FIN Flag Cnt,141
12,Bwd PSH Flags,131
1,SYN Flag Cnt,123
17,PSH Flag Cnt,116
4,ACK Flag Cnt,98
8,Down/Up Ratio,97
9,Flow Pkts/s,86
21,Init Bwd Win Byts,82


In [35]:
freq_df[:38].to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/cnn_top24_InSDN.csv', index=False)

# Detect normal samples

In [36]:
whitelist = freq_df.iloc[:, 0][:38].tolist()
print(whitelist)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'Idle Mean', 'Bwd Seg Size Avg', 'Pkt Len Mean', 'Flow IAT Mean', 'Pkt Len Std', 'TotLen Bwd Pkts', 'Bwd Pkt Len Std', 'Bwd Header Len', 'Subflow Bwd Pkts', 'Fwd Act Data Pkts', 'Fwd Pkt Len Min', 'Pkt Len Min']


In [37]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:6


In [38]:
whitelist = freq_df.iloc[:, 0][:24].tolist()
print(whitelist)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max']


In [39]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:24


## False positive of XAI on normal samples of trainset:
top 38: 6/150

top 24: 24/150